# Lab 7c: Simple Games - Traveler’s Dilemma
The Traveler’s Dilemma is a non-cooperative __non-zero sum game__ that involves an airline losing two identical suitcases belonging to two travelers. The airline requests that the travelers report the value of their suitcases, which must fall in the range of `2 USD` and `100 USD`, in increments of $\pm$ `1 USD`.

__Rules__:
* If both travelers report the same value, they receive that value as a reward. 
* However, if the travelers report different values, the traveler with the lower value receives their reported value plus an additional `2 USD`. In comparison, the traveler with the higher value receives the lower value minus `2 USD`. 

The reward function is determined as follows:

$$
\begin{eqnarray*}
R_{i}(a_{i},a_{-i}) = 
\begin{cases}
a_{i} & \text{if } a_{i} = a_{-i} \\
a_{i} + 2 & \text{if } a_{i} < a_{-i} \\
a_{-1} - 2 & \text{otherwise}
\end{cases}
\end{eqnarray*}
$$

Most people put down between `97 USD` and `100 USD`. However, somewhat counter-intuitively, there is a unique [Nash equilibrium](https://en.wikipedia.org/wiki/Nash_equilibrium) of only `2 USD`.

### Learning objectives
The objective of `Lab 7c` is to familiarize students with the [Traveler’s Dilemma problem](https://en.wikipedia.org/wiki/Traveler%27s_dilemma), the solution of the problem using iterative refinement and the concept of [Nash Equilibrium](https://en.wikipedia.org/wiki/Nash_equilibrium).

* The [Traveler’s Dilemma problem](https://en.wikipedia.org/wiki/Traveler%27s_dilemma) was first posed by [Kaushik Basu](https://en.wikipedia.org/wiki/Kaushik_Basu), a Cornell professor and former Chief Economist of the World Bank (2012 - 2016). For more information on this problem (beyond what is described here), check out this [article](https://www.academia.edu/56129718/The_Travelers_Dilemma). This problem (as we shall see) has a [Nash Equilibrium solution](https://en.wikipedia.org/wiki/Nash_equilibrium) of `2 USD`.
* In [Nash Equilibrium](https://en.wikipedia.org/wiki/Nash_equilibrium), each player is assumed to know the equilibrium strategies of the other players, and no single player can gain by changing only their strategy.

#### Sources
The implementation of the Traveler’s Dilemma problem was taken from the `Decisions` book:

* [Algorithms For Decision Making, Kochenderfer, Wheeler, Wray, MIT Press, 2022](https://algorithmsbook.com)

We've implemented some of the codes found in `Chapter 24` of the `Decisions` book in our package [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git).

## Setup
The computations in this lab (or example) are enabled by the [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5760-Labs-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`


## Task 1: Build a `MySimpleGameModel` instance

#### Model
Let's begin `Lab 7c` by constructing a model of the game, which is an instance of the `MySimpleGameModel` type:

```julia
# The game model holds values (and functions) that are useful to evaluate the game
mutable struct MySimpleGameModel <: AbstractGameModel

    # data -
    γ   # discount factor -
    ℐ   # set of players -
    𝒜   # joint action space
    R   # joint reward function

    # # constructor -
    MySimpleGameModel() = new();
end
```

Instances of the `MySimpleGameModel` type have the following fields:

* The `γ::Float64` field holds the discount factor for the game (the weight of current versus future rewards, not used in this game).
* The `ℐ::Array{Int64,1}` field holds the list of players, in our case `{1,2}`
* The `𝒜` field holds the joint action space for the game (the collection of actions for each player)
* The `R::Function` field holds the joint reward function $R(a)$: this method is called with a joint action $a$ and returns a reward.

#### Build
We build a game model by passing the type of game we want to construct, in this case a `MyTravelersProblem`, into a the `build(...)` method:

```julia
function build(simpleGame::Type{MyTravelersProblem})

    # build an empty model -
    model = MySimpleGameModel();
    
    # populate the model -
    model.γ = 0.9;
    model.ℐ = vec(collect(1:n_agents(simpleGame)))
    model.𝒜 = [ordered_actions(simpleGame, i) for i in 1:n_agents(simpleGame)]
    model.R = (a) -> joint_reward(simpleGame, a)

    # return the model -
    return model;
end
```

The `build(...)` method constructs an empty model, then populates the model with the required data. Save your instance of the game model in the `mysimplemodel` variable:

In [2]:
mysimplemodel = build(MyTravelersProblem);

## Task 2: Compute the iterated best policy for the Traveler’s Dilemma problem

The iterated best policy is computed using the `solve(...)` method, given by:

```julia
function solve(M::MyIteratedBestResponsePolicy, 𝒫::MySimpleGameModel)
    π = M.π
    for _ in 1:M.k_max
        π = [best_response_policy(𝒫, π, i) for i in 𝒫.ℐ]
    end
    return π
end
```

The `solve(...)` method takes the `MyIteratedBestResponsePolicy` and `MySimpleGameModel` instances and returns the best response policy for the Traveler’s Dilemma problem, computed by iterative refinement. The updates continue for `k_max` iterations, where during each iteration:

* The joint policy $\pi$ is updated for each player $i\in\left\{1,2\right\}$ using an [array comprehension](https://docs.julialang.org/en/v1/manual/arrays/#man-comprehensions) operation. The update calls the `best_response_policy(...)` function, which returns the deterministic best policy.
* After `k_max` updates, the refined joint policy $\pi$ is returned to the caller.

### Initialize a uniformly random policy
To construct an initial policy, we use the `build(...)` method to build and initialize an `MyIteratedBestResponsePolicy` instance. The `build(...)` method is given by:

```julia
function build(𝒫::MySimpleGameModel, k_max)
    π = [MySimpleGamePolicy(ai => 1.0 for ai in 𝒜i) for 𝒜i in 𝒫.𝒜]
    return MyIteratedBestResponsePolicy(k_max, π)
end
```

The `build(...)` method takes a `MySimpleGameModel` instance, and a value for the `k_max` parameter and returns a `MyIteratedBestResponsePolicy` with a uniform policy, which is stored in the `initial_iterated_policy` variable:

In [26]:
k_max = 100;
initial_iterated_policy = build(mysimplemodel, k_max);

### Solve
Now that we have a `initial_iterated_policy`, and the `mysimplemodel`, we can solve the problem using the `solve(...)` method shown above. The `solve(...)` method iteratively updates the initial policy and returns the `updated_policy` variable:

In [27]:
updated_policy = VLDecisionsPackage.solve(initial_iterated_policy, mysimplemodel)

2-element Vector{MySimpleGamePolicy}:
 MySimpleGamePolicy(Dict(2 => 1.0))
 MySimpleGamePolicy(Dict(2 => 1.0))

## Hmmm. Something interesting
The `solve(...)` returns the correct [Nash equilibrium](https://en.wikipedia.org/wiki/Nash_equilibrium) value, but what happens if we use another approach, for example the `best_response_policy(...)` method:

```julia
function best_response_policy(𝒢::MySimpleGameModel, π,i)
    U(ai) = utility(𝒢, joint(π, MySimpleGamePolicy(ai), i), i)
    ai = argmax(U, 𝒢.𝒜[i])
    return MySimpleGamePolicy(ai)
end
```

Does this return the correct value? To explore this question, let's initialize a uniform joint policy $\pi$:

In [ ]:
π = [MySimpleGamePolicy(ai => 1.0 for ai in 𝒜i) for 𝒜i in mysimplemodel.𝒜];

and then call the `best_response_policy(...)` method with the `mysimplemodel` instance, the joint policy $\pi$ and the index of the player $i$. Save the result in the `best` variable:

In [17]:
i = 2;
best = best_response_policy(mysimplemodel, π, i)

MySimpleGamePolicy(Dict(97 => 1.0))